In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
import mysql.connector

# inisialisasi array untuk menampilkan hasil
id_detail = []
deskripsi_lowongan = []
stam_detail = []

# konektor ke database
db = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='UnNgGrape'
        )
cursor = db.cursor()

cursor.execute("SELECT id_jobstreet, link_lowongan FROM jobstreet")
raw_jobstreet = cursor.fetchall()
for row in raw_jobstreet:
    id_jobstreet = row[0]
    link = row[1]

    r = Request(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
    response = urlopen(r).read()
    soup = BeautifulSoup(response, "lxml")

    raw_detail = soup.find_all("div","sx2jih0 qs0rX_0 _19ukm_0")
    for p in raw_detail:
        # menggunakan try except karena ada beberapa deskripsi yang NoneType
        try:
            raw_deskripsi = p.find("div", "sx2jih0 h6p8rp0 h6p8rp7 h6p8rph" ).get_text(separator=". ").replace("Deskripsi Pekerjaan.", "")
            nltk_tokens = nltk.sent_tokenize(raw_deskripsi)
            deskripsi = ""

            for x in nltk_tokens:
                if x == "Tentang Perusahaan .":
                    break
                deskripsi = deskripsi + " " + x
        except:
            deskripsi = "Eror: deskripsi Nonetype"

        raw_stam = deskripsi
        # create stemmer
        # nltk
        ps = PorterStemmer()
        # sastrawi
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()

        # stemming process
        # nltk
        words = word_tokenize(raw_stam)
        nltk_stammer = ""
        for w in words:
            nltk_stammer = nltk_stammer + " " + ps.stem(w)
        # sastrawi
        output_stammer = stemmer.stem(nltk_stammer)
            
        # mengecek apakah data sudah adat atau belum
        cursor.execute("SELECT id_jobstreet FROM detail_jobstreet WHERE id_jobstreet = '" + str(id_jobstreet) + "'")
        data = cursor.fetchall()
        # if else untuk pengecekan input data
        if data:
            print("Data exists")
        else:
            cursor.execute(
                        "INSERT INTO detail_jobstreet(id_jobstreet, deskripsi_lowongan, stam_detail)"
                        "VALUES ('"+ str(id_jobstreet) +"', '"+ deskripsi.replace("'", "").replace('"', '') +"', '"+ output_stammer +"')"
                        )
            
            db.commit()
            print("Data has been added")

        id_detail.append(id_jobstreet)
        deskripsi_lowongan.append(deskripsi)
        stam_detail.append(output_stammer)

detail_dict ={'id':id_detail, 'deskripsi':deskripsi_lowongan, 'stam':stam_detail}
df = pd.DataFrame(detail_dict, columns = ['id', 'deskripsi', 'stam'])

df.sort_values('id', ascending=True)

Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host